The physics of an `Interchange` is included in a sequence of objects called potential handlers.

For starters, let's create the same single-molecule `Interchange` we did in the `Exports.ipynb` notebook.

In [ ]:
from openff.toolkit import ForceField, Molecule

from openff.interchange import Interchange

molecule = Molecule.from_smiles("c1ccccc1-c2ccccc2")
molecule.generate_conformers(n_conformers=1)

sage = ForceField("openff-2.0.0.offxml")

interchange = Interchange.from_smirnoff(sage, molecule.to_topology())

Let's quickly visualize this molecule with atom indices. This will be useful if later on we want to look up particular parameters. Note that this indexes the atoms at 1 whereas OpenFF indexed at 0.

In [ ]:
from rdkit.Chem import Mol as RDMol


# Adapted from https://www.rdkit.org/docs/Cookbook.html#include-an-atom-index
def mol_with_atom_index(molecule: Molecule):
    molecule_copy = Molecule(molecule)
    molecule_copy._conformers = None

    rdmol: RDMol = molecule_copy.to_rdkit()
    for atom in rdmol.GetAtoms():
        atom.SetAtomMapNum(atom.GetIdx())

    return rdmol


mol_with_atom_index(molecule)

The potential handlers are stored as a dictionary in the `Interchange.handlers` attribute as a `Dict[str, PotentialHandler]` mapping.

In [ ]:
[(key, type(value)) for key, value in interchange.handlers.items()]

Recall that SMIRNOFF groups force field parameters into groups ("handlers") roughly corresponding to terms in the potential energy function. These all correspond to components in an `Interchange` object, though not necessarily all 1:1.

In [ ]:
sage.registered_parameter_handlers

Each potential handler implements a few key methods as required by the base class. These are
* `type`: A string identifying the type of stored potentials
* `expression`: An algebraic expression (or otherwise information) used to compute the potential energy from this handler
* `supported_paramters`: A sequence of strings identifying the parameters supported by this handler (i.e. `k`, `periodicity`, `phase`)
* `slot_map`: A mapping between topological locations ("slots") and unique identifiers of applied parameters
* `potentials`: A mapping between unique identifiers of applies parameters and the parameters themselves.

Each handler may also introduce more fields and methods as needed.

In [ ]:
from openff.interchange.components.smirnoff import SMIRNOFFPotentialHandler

SMIRNOFFPotentialHandler?

As a simple case, let's look at the bond handler and its contents.

In [ ]:
bond_handler = interchange["Bonds"]

bond_handler.type, bond_handler.expression

In [ ]:
bond_handler.fractional_bond_order_interpolation, bond_handler.fractional_bond_order_method

In [ ]:
bond_handler.slot_map

Let's inspect these objects one  by one, starting with the first topology key in the slot map.

In [ ]:
first_topology_key = [*bond_handler.slot_map.keys()][0]

first_topology_key, type(first_topology_key)

This object stores information about where in the topology some parameter is meant to be found. In this case, that is fully specified by the indices of the two atoms in the bonds.

In [ ]:
first_topology_key.atom_indices

The `PotentialHandler.slot_map` maps from these keys to `PotentialKey` objects, which are unique identifiers of parameters.

In [ ]:
first_potential_key = bond_handler.slot_map[first_topology_key]

first_potential_key, type(first_potential_key)

In the case of SMIRNOFF force fields, the SMIRKS pattern uniquely identifies the parameters in a particular handler. In other typing schemes this might be an atom type or a combination of atom types.

In [ ]:
first_potential_key.id, first_potential_key.associated_handler

Finally, `PotentialHandler` maps from these potential keys to `potential` objects themselves, allowing for parameter de-duplication and quick lookup.

In [ ]:
first_potential = bond_handler.potentials[first_potential_key]

first_potential, type(first_potential)

In [ ]:
first_potential.parameters

Putting this all together, one could write a function that takes in two atom indices and returns the equilibrium bond length and then use this function to compare the parameters applied to carbon-carbon bonds in each ring and between them.

In [ ]:
from openff.interchange.models import PotentialKey, TopologyKey


def get_r(interchange: Interchange, atom_indices: tuple[int]):
    bond_handler = interchange["Bonds"]

    topology_key = TopologyKey(atom_indices=atom_indices)
    potential_key = bond_handler.slot_map[topology_key]
    potential = bond_handler.potentials[potential_key]

    return potential.parameters["length"]


get_r(interchange, (0, 1)), get_r(interchange, (5, 6))

More interestingly, we could modify this slightly to instead compare the force constant of in-ring and between-ring torsions. (This only reports the force constant of one phase, but these torsions each happen to be single-phase.)

In [ ]:
def get_k(interchange: Interchange, atom_indices: tuple[int]):
    bond_handler = interchange["ProperTorsions"]

    topology_key = TopologyKey(atom_indices=atom_indices, mult=0)
    potential_key = bond_handler.slot_map[topology_key]
    potential = bond_handler.potentials[potential_key]

    return potential.parameters["k"]


get_k(interchange, (0, 1, 2, 3)), get_k(interchange, (0, 5, 6, 7))